In [1]:
import copy
from pprint import pprint
from collections import deque

In [2]:
def SteadyRouter(rid):
    global routers

    print("==== Steadying Router:%d ===="%(rid))
    # LOOP OVER ALL INPUT AND OUTPUT PORTS
    
    # Calculate λj for each output port
    for j in range(routers[rid]['num_ports']):
        routers[rid][j]['agg_output_arrival_rate'] = 0
        for i in range(routers[rid]['num_ports']):
            routers[rid][j]['agg_output_arrival_rate'] += routers[rid]['queue_arrival_rate'][i][j]
    
    # Calculate μj for each ouput port
    # - This is similar to calculating λj, but μj cannot exceed the output port bandwidth
    for j in range(routers[rid]['num_ports']):
        output_port_service_rate = 0
        for i in range(routers[rid]['num_ports']):
            output_port_service_rate += routers[rid]['queue_arrival_rate'][i][j]
        routers[rid][j]['agg_output_service_rate'] = min(output_port_service_rate, link_transmission_rate)

    # Get max shared rate by counting number of i->j transfers for each output port
    #output_port_max_shared_rate = {}
    output_port_active_inputs = {}
    for j in range(routers[rid]['num_ports']):
        output_port_active_inputs[j] = set()
        for i in range(routers[rid]['num_ports']):
            if routers[rid]['queue_arrival_rate'][i][j] > 0:
                output_port_active_inputs[j].add(i)

        #output_port_max_shared_rate[j] = routers[rid][j]['agg_output_service_rate']/output_port_active_transfers[j]

    # Store old arrival rates and clear service rates
    for i in range(routers[rid]['num_ports']):
        for j in range(routers[rid]['num_ports']):
            # Store old rates
            routers[rid]['queue_arrival_rate_old'][i][j] = routers[rid]['queue_arrival_rate'][i][j]
            routers[rid]['queue_service_rate'][i][j] = 0
    
    # Calculate μi→j based on equal sharing of μj
    for j in range(routers[rid]['num_ports']):
        if routers[rid][j]['agg_output_arrival_rate'] > 0:
            
            remaining_bw = routers[rid][j]['agg_output_service_rate']
            while (remaining_bw > 0):
                output_port_max_shared_rate = remaining_bw/len(output_port_active_inputs[j])
                
                for i in range(routers[rid]['num_ports']):
                    # Only check ports with active arrivals
                    if i not in output_port_active_inputs[j]:
                        continue

                    unsatified_demand = routers[rid]['queue_arrival_rate'][i][j] \
                                        -routers[rid]['queue_service_rate'][i][j]
                    
                    # if demand (λi-j) is less than or equal to alloted share of μj, allocate & satify demand
                    if unsatified_demand <= output_port_max_shared_rate:
                        routers[rid]['queue_service_rate'][i][j] += unsatified_demand
                        remaining_bw -= unsatified_demand
                        output_port_active_inputs[j].remove(i)

                    # if demand (λi-j) greater than alloted share of μj, assign allotment
                    if unsatified_demand > output_port_max_shared_rate:
                        routers[rid]['queue_service_rate'][i][j] += output_port_max_shared_rate
                        remaining_bw -= output_port_max_shared_rate


    # Balance arrivals and service rates
    for i in range(routers[rid]['num_ports']):
        for j in range(routers[rid]['num_ports']):
            # Group 1: same arrival and service rate
            # - do nothing

            # Group 2: arrival rate less than service rate
            if routers[rid]['queue_arrival_rate'][i][j] < routers[rid]['queue_service_rate'][i][j]:
                # - This should not happen. Print error
                print("=-=-=-=-=- Error in SteadyRouter: rate imbalance =-=-=-=-=-")

            # Group 3: arrival rate more than service rate; reduce arrival rate
            else:
                routers[rid]['queue_arrival_rate'][i][j] = routers[rid]['queue_service_rate'][i][j]

    
    # Update aggregate input rates
    for i in range(routers[rid]['num_ports']):
        routers[rid][i]['agg_input_arrival_rate'] = 0
        routers[rid][i]['agg_input_service_rate'] = 0
        for j in range(routers[rid]['num_ports']):
            routers[rid][i]['agg_input_arrival_rate'] += routers[rid]['queue_arrival_rate'][i][j]
            routers[rid][i]['agg_input_service_rate'] += routers[rid]['queue_service_rate'][i][j]


In [3]:
# Update the arrival rate for all cross-flows through the router
def UpdateArrival(rid):
    global routers
    
    for i in range(routers[rid]['num_ports']):
        for j in range(routers[rid]['num_ports']):
            routers[rid]['queue_arrival_rate_old'][i][j] = 0 #routers[rid]['queue_arrival_rate'][i][j]
            
            routers[rid]['queue_arrival_rate'][i][j] = 0
            for flow_id in routers[rid][i][j]['flow_ids']:
                routers[rid]['queue_arrival_rate'][i][j] += min(flow_table[flow_id]['rate'], flow_table[flow_id]['demand'])

        routers[rid][i]['agg_input_arrival_rate'] = 0
        for j in range(routers[rid]['num_ports']):
            routers[rid][i]['agg_input_arrival_rate'] += routers[rid]['queue_arrival_rate'][i][j]


In [4]:
# Initialize router queue occupancies
def InitializeRouters(routers_to_process):
    global routers
    global flow_table
    global occupancy

    for rid in list(routers_to_process):
        UpdateArrival(rid)
        
        # Initialize input queuing occupancy
        if occupancy and rid in occupancy:                    
            for (i_j_tuple), initial_occ in occupancy[rid].items():
                (i, j) = i_j_tuple 
                if routers[rid][i]['agg_input_arrival_rate'] > 0:
                    routers[rid]['queue_occupancy'][i][j] =  initial_occ
                else:
                    routers[rid]['queue_occupancy'][i][j] = 0
        else:        
            for i in range(routers[rid]['num_ports']):
                for j in range(routers[rid]['num_ports']):
                    if routers[rid][i]['agg_input_arrival_rate'] > 0:
                        routers[rid]['queue_occupancy'][i][j] = \
                            (routers[rid]['queue_arrival_rate'][i][j] / routers[rid][i]['agg_input_arrival_rate']) * \
                                routers[rid]['queue_occupancy'][i][j]
                    else:
                        routers[rid]['queue_occupancy'][i][j] = 0

        for j in range(routers[rid]['num_ports']):
            output_port_arrival_rate = 0
            for i in range(routers[rid]['num_ports']):
                output_port_arrival_rate += routers[rid]['queue_arrival_rate'][i][j]
            
            routers[rid][j]['agg_output_service_rate'] = min(output_port_arrival_rate, link_transmission_rate)
'''
        for j in range(routers[rid]['num_ports']):
            if routers[rid][j]['agg_output_service_rate'] > 0:
                # Add the router if there’s flow through it
                routers_to_process.append(rid)
                break
    
    #return routers_to_process
'''                

"\n        for j in range(routers[rid]['num_ports']):\n            if routers[rid][j]['agg_output_service_rate'] > 0:\n                # Add the router if there’s flow through it\n                routers_to_process.append(rid)\n                break\n    \n    #return routers_to_process\n"

In [5]:
def SteadyNetwork(routers_to_process):
    global routers
    global flow_table

    # Repeat until all routers reach equilibrium
    while len(routers_to_process) > 0:
        rid = routers_to_process.pop()
        
        UpdateArrival(rid)
        SteadyRouter(rid)
        
        for i in range(routers[rid]['num_ports']):
            for j in range(routers[rid]['num_ports']):
                if routers[rid]['queue_arrival_rate'][i][j] != routers[rid]['queue_arrival_rate_old'][i][j]:
                    
                    #if routers[rid]['queue_arrival_rate_old'][i][j] > 0:
                    rate_multiplier =  routers[rid]['queue_arrival_rate'][i][j]/routers[rid]['queue_arrival_rate_old'][i][j]

                    for flow_id in routers[rid][i][j]['flow_ids']:
                        # Update rate of all crossing flows
                        flow_table[flow_id]['rate'] = min(rate_multiplier * flow_table[flow_id]['rate'], \
                                                          flow_table[flow_id]['demand'])
                        
                        # Reconsider routers along the flow path
                        for flow_router in flow_table[flow_id]['routers']:
                            if flow_router != rid:
                                if flow_router not in routers_to_process:
                                    routers_to_process.appendleft(flow_router)

                    routers[rid]['queue_arrival_rate_old'][i][j] = routers[rid]['queue_arrival_rate'][i][j]

In [6]:
### DUMBBELL network and flows
### This section contains the inputs to the model and variables that must be initilized before steadying the network

link_transmission_rate = 25
router_queue_capacity = 8448
router_num_ports = 3
rids = [2, 3]

'''
node0                                     node5
 ┌─┐                                      ┌─┐  
 └─┘                                      └─┘  
  │      ┌──────────┐      ┌──────────┐    ▲   
  └─────►│0         │      │         2├────┘   
         │  node2  2├─────►│0  node3  │        
  ┌─────►│1         │      │         1├────┐   
  │      └──────────┘      └──────────┘    ▼   
 ┌─┐                                      ┌─┐  
 └─┘                                      └─┘  
node1                                     node4
'''

# Initialize table of flows and their properties 
flow_table = {0: {'source': 0, 'destination': 5, 'demand': 20, 'rate': 20, 'routers': []},
              1: {'source': 1, 'destination': 4, 'demand': 15, 'rate': 15, 'routers': []}}


# Static routing table for finding path through the network
routing_table = {0:          # nodeid
                     {5: (0, 0, 2)}, # dst : (curr_output_port, next_input_port, next_node)
                 1:
                     {4: (0, 1, 2)},
                 2:
                     {4: (2, 0, 3),
                      5: (2, 0, 3)},
                 3:
                     {4: (1, 0, 4),
                      5: (2, 0, 5)},
                }
'''
# this is Q_i
occupancy = {
    2: {            # router: {(input port, output port): occupancy, }
        (0, 1): 0,  # q_(i→j)
        (0, 2): 5,
        (1, 0): 4,
        (1, 1): 3,
        (1, 2): 1,
        },
    3: { 
        (0, 0): 0,
        (0, 1): 5,
        (0, 2): 5,
        }
}
'''
occupancy = {}

In [7]:


# Define base router
baserouter = {'num_ports' : router_num_ports,
          'queue_arrival_rate': [[]] * router_num_ports,   # λ_(i→j)'
          'queue_service_rate': [[]] * router_num_ports,   # μ_(i→j)
          'queue_occupancy'   : [[]] * router_num_ports,   # q_(i→j)
          'queue_arrival_rate_old': [[]] * router_num_ports, 
         }
for i in range(baserouter['num_ports']): # each input port
    baserouter['queue_arrival_rate'][i] = [0] * router_num_ports
    baserouter['queue_service_rate'][i] = [0] * router_num_ports
    baserouter['queue_occupancy'][i] = [0] * router_num_ports
    baserouter['queue_arrival_rate_old'][i] = [0] * router_num_ports
    
    baserouter[i] = {} 
    baserouter[i]['agg_input_arrival_rate']  = 0   # λ_i
    baserouter[i]['agg_input_service_rate']  = 0   # μ_i
    baserouter[i]['agg_output_arrival_rate'] = 0   # λ_j
    baserouter[i]['agg_output_service_rate'] = 0   # μ_i
    baserouter[i]['queue_capacity'] = router_queue_capacity   # Q_i
    for j in range(baserouter['num_ports']):
        baserouter[i][j] = {'flow_ids': []} # list of flows this path through the router


# Initialize all router nodes
routers = {}
for rid in rids:
    routers[rid] = copy.deepcopy(baserouter)

# Add flows to routers and routers to flow table
for flow_id in flow_table.keys():
    # Read flow information
    rate = flow_table[flow_id]['demand']
    src = flow_table[flow_id]['source']
    dst = flow_table[flow_id]['destination']

    # Walk through the nodes along the flow's routed path
    (curr_output_port, next_input_port, next_node) = routing_table[src][dst]
    while next_node != dst:
        next_output_port = routing_table[next_node][dst][0]
        
        # Add flow to router input port arrivals
        #routers[next_node]['queue_arrival_rate'][next_input_port][next_output_port] += rate
        #routers[next_node]['queue_service_rate'][next_input_port][next_output_port] += rate
        routers[next_node][next_input_port][next_output_port]['flow_ids'].append(flow_id)

        # Add router to flow table
        flow_table[flow_id]['routers'].append(next_node)
        
        # Move to next hop
        (curr_output_port, next_input_port, next_node) = routing_table[next_node][dst]

routers_to_start = deque([])
routers_to_start.appendleft(flow_table[0]['routers'][0])

In [8]:
# Please run the previous to initialize the network before running this cell

#InitializeRouters(routers_to_start)
SteadyNetwork(routers_to_start)

print("\n=== Steadying Network complete===========================================")
#print("\n=== Routers ===")
#pprint(routers)
print("\n=== Flow table: ===")
pprint(flow_table)


==== Steadying Router:2 ====
==== Steadying Router:3 ====

=== Steadying Network complete===========================================

=== Flow table: ===
{0: {'demand': 20,
     'destination': 5,
     'rate': 12.5,
     'routers': [2, 3],
     'source': 0},
 1: {'demand': 15,
     'destination': 4,
     'rate': 12.5,
     'routers': [2, 3],
     'source': 1}}
